## Controle sua planta via MQTT

Até agora, seu relé é controlado pelo dispositivo IoT diretamente com base em uma única leitura de umidade do solo. Em um sistema de irrigação comercial, a lógica de controle será centralizada, permitindo que ele tome decisões sobre irrigação usando dados de vários sensores e permitindo que qualquer configuração seja alterada em um único lugar. Para simular isso, você pode controlar o relé por MQTT.


### Tarefa - controlar o relé sobre MQTT

1. Adicione as bibliotecas/pacotes pip e o código MQTT relevantes ao seu projeto `soil-moisture-sensor` para conectar-se ao MQTT. Nomeie o ID do cliente como `soilmoisturesensor_client` prefixado pelo seu ID.

    > ⚠️ Você pode consultar [as instruções para conectar ao MQTT no projeto 1, lição 4, se necessário](https://github.com/microsoft/IoT-For-Beginners/blob/main/1-getting-started/lessons/4-connect-internet/README.md#connect-your-iot-device-to-mqtt).

1. Adicione o código do dispositivo relevante para enviar telemetria com as configurações de umidade do solo. Para a mensagem de telemetria, nomeie a propriedade `soil_moisture`.

    > ⚠️ Você pode consultar [as instruções para enviar telemetria para MQTT no projeto 1, lição 4, se necessário](https://github.com/microsoft/IoT-For-Beginners/blob/main/1-getting-started/lessons/4-connect-internet/README.md#send-telemetry-from-your-iot-device).

1. Crie algum código de servidor local para assinar a telemetria e envie um comando para controlar o relé em uma pasta chamada `soil-moisture-sensor-server`. Nomeie a propriedade na mensagem de comando `relay_on` e defina o ID do cliente como `soilmoisturesensor_server` prefixado pelo seu ID. Mantenha a mesma estrutura do código do servidor que você escreveu para o projeto 1, lição 4, pois você adicionará a este código mais tarde nesta lição.

    > ⚠️ Você pode consultar [as instruções para enviar telemetria para MQTT](https://github.com/microsoft/IoT-For-Beginners/blob/main/1-getting-started/lessons/4-connect-internet/README.md#write-the-server-code) e [enviar comandos por MQTT](https://github.com/microsoft/IoT-For-Beginners/blob/main/1-getting-started/lessons/4-connect-internet/README.md#send-commands-to-the-mqtt-broker) no projeto 1, lição 4, se necessário.

1. Adicione o código do dispositivo relevante para controlar o relé a partir de comandos recebidos, usando a propriedade `relay_on` da mensagem. Envie true para `relay_on` se `soil_moisture` for maior que 450, caso contrário, envie false, o mesmo que a lógica que você adicionou para o dispositivo IoT anteriormente.

    > ⚠️ Você pode consultar [as instruções para responder aos comandos do MQTT no projeto 1, lição 4, se necessário](https://github.com/microsoft/IoT-For-Beginners/blob/main/1-getting-started/lessons/4-connect-internet/README.md#handle-commands-on-the-iot-device).

> 💁 Você pode encontrar este código na pasta [code-mqtt](https://github.com/microsoft/IoT-For-Beginners/blob/main/2-farm/lessons/3-automated-plant-watering/code-mqtt).

Certifique-se de que o código esteja sendo executado no seu dispositivo e no servidor local e teste-o alterando os níveis de umidade do solo, alterando os valores enviados pelo sensor virtual ou alterando os níveis de umidade do solo adicionando água ou removendo o sensor do solo.


# Veja os códigos 2.1 sensor e 2.2 servidor

In [1]:
import time
import json
import random
from counterfit_connection import CounterFitConnection
from counterfit_shims_grove.adc import ADC
from counterfit_shims_grove.grove_relay import GroveRelay
import paho.mqtt.client as mqtt
CounterFitConnection.init('127.0.0.1', 5001)

adc = ADC()

relay = GroveRelay(5)


# Configuração do broker MQTT
broker = 'mqtt.eclipseprojects.io'  # Endereço do broker MQTT
port = 1883  # Porta padrão para conexões MQTT
telemetry_topic = "capacitacao-iot/telemetria"  # Tópico de telemetria para envio de dados
commands_topic = "capacitacao-iot/commandos" # tópico onde serão recebidos os comandos

# Gera um identificador único para o cliente MQTT
client_id = f'capacitacao-iot-mqtt-nightlight_client-{random.randint(0, 100000)}'
print("Client ID: " + client_id)

# Função de callback chamada ao conectar-se ao broker MQTT
def on_connect(client, userdata, flags, reason_code, properties):
    # Verifica se a conexão foi bem-sucedida ou houve falha
    if reason_code != 0:
        print(f"Falha ao conectar: {reason_code}. 'loop_forever()' tentará reconectar.")
    else:
        print("Conectado ao broker MQTT!")
        mqttc.subscribe(commands_topic)


# agora, o código do dispositivo também deverá reagir a eventos
# Por isso, ao receber uma mensagem, o dispositivo deve reagir ao que foi comandado. 
def handle_command(client, userdata, message):
    payload = json.loads(message.payload.decode())
    print("Command received:", payload)

    if payload['relay_on']:
        relay.on()
    else:
        relay.off()

# Cria o cliente MQTT e configura a função de callback
mqttc = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2, client_id=client_id)
mqttc.on_connect = on_connect  # Associa a função de callback ao evento de conexão
mqttc.subscribe(commands_topic, qos=1)
mqttc.on_message = handle_command
mqttc.user_data_set([])  # Configura dados de usuário, se necessário

# Conecta ao broker MQTT e inicia o loop de comunicação
mqttc.connect(broker, port)
mqttc.loop_start()

# Aguarda um tempo inicial para garantir que a conexão esteja estabelecida
time.sleep(1)


while True:
    soil_moisture = adc.read(0)
    print("Soil moisture:", soil_moisture)

    # Formata a leitura do sensor em JSON para envio
    soil_moisture_telemetry = json.dumps({"soil_moisture": soil_moisture})
    
    # Envia os dados ao broker MQTT
    print("Enviando JSON:", soil_moisture_telemetry, "no tópico:", telemetry_topic)
    mqttc.publish(telemetry_topic, soil_moisture_telemetry)
    
    time.sleep(10)

Client ID: capacitacao-iot-mqtt-nightlight_client-69513
Conectado ao broker MQTT!
Soil moisture: 321
Enviando JSON: {"soil_moisture": 321} no tópico: capacitacao-iot/telemetria
Soil moisture: 61
Enviando JSON: {"soil_moisture": 61} no tópico: capacitacao-iot/telemetria
Soil moisture: 108
Enviando JSON: {"soil_moisture": 108} no tópico: capacitacao-iot/telemetria
Command received: {'relay_on': True}
Soil moisture: 785
Enviando JSON: {"soil_moisture": 785} no tópico: capacitacao-iot/telemetria
Command received: {'relay_on': False}


KeyboardInterrupt: 